In [ ]:
!pip install sdv==0.18.0

In [ ]:
from sdv.timeseries import PAR
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from pickle import dump, load

In [ ]:
# https://www.kaggle.com/code/xiaxiaxu/predictmachinefailureinadvance/data
sensor = pd.read_csv("sensor.csv")

# only keeping cols w high var in pca analysis + machine status
data = sensor[
    ["timestamp", "sensor_25", "sensor_11", "sensor_36", "sensor_34", "machine_status"]
]

In [ ]:
data_around_failures = data.iloc[16000:26000]
data_around_failures.head()

In [ ]:
data_around_failures["timestamp"] = pd.to_datetime(data_around_failures["timestamp"])

In [ ]:
print(data_around_failures[data_around_failures.isna().any(axis=1)])

# we only have one row with NA, let's simply drop that.

data_around_failures.dropna(axis=0, inplace=True)

print(data_around_failures[data_around_failures.isna().any(axis=1)])

In [ ]:
sensor_25_data = data_around_failures[["timestamp", "sensor_25"]]
sensor_25_data.head()

In [ ]:
import matplotlib.pyplot as plt

sensor_25_data.set_index("timestamp").plot()

In [ ]:
sensor_25_scaler = MinMaxScaler()

# scaling our data, then saving our scaler object for future use.
sensor_25_data[["sensor_25"]] = sensor_25_scaler.fit_transform(
    sensor_25_data[["sensor_25"]]
)

dump(sensor_25_scaler, open("sensor_25_scaler.pkl", "wb"))

In [ ]:
print(sensor_25_data.dtypes)
print(len(sensor_25_data))

sensor_25_data.head()

In [ ]:
# okay, we should be set to train our first model. let's give it a go!

sequence_index = "timestamp"
sensor_25_model = PAR(sequence_index=sequence_index, verbose=True, epochs=48)

In [ ]:
sensor_25_model.fit(sensor_25_data)

In [ ]:
sensor_25_model.save("25_model.pkl")

In [ ]:
# sample = sensor_25_model.sample(num_sequences=1)

In [ ]:
# sample.set_index('timestamp').plot()